## SpaceX Dash

In [ ]:
!pip install jupyter-dash

In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-4-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-4-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [6]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("07_spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',options=[
                                    {'label':'All Sites', 'value':'ALL'},
                                    {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                    {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                    {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                    {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'}
                                ],
                                value= 'ALL',
                                placeholder='Select a Launch Site here',
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload]),
                                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])




In [8]:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',component_property='figure'),
                Input(component_id='site-dropdown',component_property='value'))
def get_pie_chart(entered_site):
    grouped_df = spacex_df.groupby(["Launch Site", "class"])["class"].count().reset_index(name="count")
    all_success=grouped_df[grouped_df['class']==1]
    if entered_site == 'ALL':
        fig = px.pie(grouped_df,values='count',
        names='Launch Site',
        title='Successful Launches from all launch sites')
        fig.update_traces(textinfo='value+percent')
        return fig
    else:
        select_site = grouped_df[grouped_df['Launch Site']==entered_site]
        select_site.sort_values(by=['class'],ascending=True)
        success=['Failure','Success']
        fig = px.pie(select_site,values='count',
        names=success,
        title=f'Launches from {entered_site} launch site')
        fig.update_traces(textinfo='value+percent')
        return 

In [9]:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def get_scatter(entered_site,slide_range):
    low,high =slide_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if entered_site == 'ALL':
        fig = px.scatter(spacex_df[mask], x="Payload Mass (kg)", y="class", 
        color="Booster Version Category",
        title='Successful launches from all sites for different Payload and Booster Version')
        return fig
    else:
        selected_site = spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.scatter(selected_site[mask], x="Payload Mass (kg)", y="class", 
        color="Booster Version Category",
        title=f'Successful launches from {entered_site} for different Payload and Booster Version')
        return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [30/May/2022 10:29:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:37] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_4_1m1653873998.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:37] "GET /_dash-component-suites/dash/deps/react@16.v2_4_1m1653873998.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:37] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_4_1m1653873998.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:37] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_4_1m1653873998.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:37] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_4_1m1653873998.min.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:38] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_4_0m1653873998.js HTTP/1.1" 200 -
127.0.0.1 - - [30/May/2022 10:29:38] "GET /_dash-component-suites/d